In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
import joblib
from scipy import sparse
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn import model_selection
from sklearn.model_selection import cross_val_score,cross_validate, KFold
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [2]:
# load data 
dataframe = pd.read_csv(r'C:\Users\dawns\Desktop\Master thesis\data\Features_level.csv')
#data=dataframe[['CONTENT_NEW','','SCORE']].copy()
df_data=dataframe[['CONTENT_NEW','WORDCUT','SCORE','LEVEL','Tokens','level1_tokens','level2_tokens','level3_tokens','level4_tokens','level5_tokens','level6_tokens','level7_tokens','char_num','high_level','primary_level','Connective_ratio','connective_SCORE','pre','connective_numbers']].copy()
df_data.head()

# Generating 'y' for classification as well as regression
y = df_data['SCORE']
#y

# generating x 
pre = df_data['pre']

In [3]:
# load your model
ngram_vector = joblib.load('ngram_vector.pkl') 

In [4]:
feature_set2 = ngram_vector

In [5]:
# 划分训练集 ，测试集 train_test split
x_train, x_test, y_train, y_test = train_test_split(feature_set2, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)


(2934, 1000)
(2934,)


In [6]:
# linear regression 线性回归
from sklearn.linear_model import LinearRegression
linear_regressor = LinearRegression()
linear_regressor.fit(x_train, y_train)
y_pred = linear_regressor.predict(x_test)
r2 = r2_score(y_test,y_pred)
neg_mean_absolute_error = mean_absolute_error(y_test,y_pred)
neg_mean_squared_error = mean_squared_error(y_test,y_pred)
print('neg_mean_absolute_error:',neg_mean_absolute_error)
print('neg_mean_squared_error:',neg_mean_squared_error)
print('r2:',r2)

neg_mean_absolute_error: 7.9568731615484065
neg_mean_squared_error: 100.82679522010855
r2: 0.24277222182076474


In [7]:
# 交叉验证 cross validation linear
cv_scores =[]
scores = cross_val_score(linear_regressor,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())

[array([ 0.04727751,  0.22055785,  0.24767033, -0.06170641,  0.16501311,
        0.05140916,  0.05939736,  0.01199606,  0.1541229 , -0.02603074])]
0.08697071332685952


In [8]:

# 随机森林 random forest

rf = ensemble.RandomForestRegressor()
params = {'n_estimators': [100], 'max_depth':[100], 'max_features':[10]}
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print('r2:',r2_score(y_test,y_pred))
print('mean_squared_error:',mean_squared_error(y_test,y_pred))
print('mean_absolute_error:',mean_absolute_error(y_test,y_pred))

r2: 0.3344740590134736
mean_squared_error: 88.61646349909174
mean_absolute_error: 7.476505449591281


In [9]:
# randomforest regressor
cv_scores =[]
scores = cross_val_score(rf,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())

[array([0.2971366 , 0.29938175, 0.30071637, 0.33634757, 0.34382737,
       0.28159808, 0.32035894, 0.28489681, 0.31750289, 0.28245211])]
0.3064218487228514


In [10]:
#xgboost  xgboost处理回归
model = xgb.XGBRegressor()
params = {'n_estimators': [160], 'learning_rate': [0.1]}
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('R2:',r2_score(y_test,y_pred))
print('mean_squared_error:',mean_squared_error(y_test,y_pred))
print('mean_absolute_error:',mean_absolute_error(y_test,y_pred))

R2: 0.3382893504518767
mean_squared_error: 88.1084477875051
mean_absolute_error: 7.45931051082767


In [11]:
#xgboost
cv_scores =[]
scores = cross_val_score(model,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())

[array([0.24066173, 0.25806655, 0.28470676, 0.27834026, 0.283613  ,
       0.20927234, 0.3088947 , 0.28543814, 0.26874066, 0.2496645 ])]
0.26673986391409077
